In [2]:
# spaCy based imports
import spacy
nlp = spacy.load('fr_core_news_sm')

In [3]:
# Usual imports 
import pandas as pd
from tqdm import tqdm, tqdm_notebook

# NLP imports
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import stop_words
import re, sys
from gensim.models import phrases, Phrases

# Sklearn imports
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
from sklearn.model_selection import GridSearchCV

# Dataviz imports
import pyLDAvis.sklearn

In [4]:
get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""
pd.set_option('display.max_colwidth', -1)

import nltk
nltk.download('punkt')

In [5]:
tqdm.pandas()
tqdm_notebook()

0it [00:00, ?it/s]

In [16]:
# Data import
answers = pd.read_csv('preprocessed/answers_lemmatized.csv', sep="\t", encoding="utf-8")
answers['theme'].unique()

,Unnamed: 0,customer_id,question_name,answer,theme,tokens,tokens_clean,tokens_clean_joined,lemmatized_answer
0,0,17 943,DC11,"Dites leur la vérité. Si une situation est jugée catastrophique c'est aux citoyen(ne)s d'y répondre et non à un gouvernement qui ne nous entends pas ! && Il y en à marre des faux semblant. Vous voulez que la Démocratie se renforce ? Ecoutez les citoyen(ne)s, cessez de faire la sourde oreille. Nous avons toutes et tous nos mots à dire pour la gestion de nos vies, comme celles de nos enfants ! Un Pays libre est un pays qui vit par et pour le Peuple !",Démocratie et citoyenneté,"['Dites', 'leur', 'la', 'vérité', '.', 'Si', 'une', 'situation', 'est', 'jugée', 'catastrophique', ""c'est"", 'aux', 'citoyen', '(', 'ne', ')', 's', 'd', ""'"", 'y', 'répondre', 'et', 'non', 'à', 'un', 'gouvernement', 'qui', 'ne', 'nous', 'entends', 'pas', '!', '&', '&', 'Il', 'y', 'en', 'à', 'marre', 'des', 'faux', 'semblant', '.', 'Vous', 'voulez', 'que', 'la', 'Démocratie', 'se', 'renforce', '?', 'Ecoutez', 'les', 'citoyen', '(', 'ne', ')', 's', ',', 'cessez', 'de', 'faire', 'la', 'sourde', 'oreille', '.', 'Nous', 'avons', 'toutes', 'et', 'tous', 'nos', 'mots', 'à', 'dire', 'pour', 'la', 'gestion', 'de', 'nos', 'vies', ',', 'comme', 'celles', 'de', 'nos', 'enfants', '!', 'Un', 'Pays', 'libre', 'est', 'un', 'pays', 'qui', 'vit', 'par', 'et', 'pour', 'le', 'Peuple', '!']","['dites', 'vérité', 'situation', 'jugée', 'catastrophique', 'citoyen', 'répondre', 'non', 'gouvernement', 'entends', 'marre', 'faux', 'semblant', 'voulez', 'démocratie', 'renforce', 'ecoutez', 'citoyen', 'cessez', 'sourde', 'oreille', 'toutes', 'mots', 'dire', 'gestion', 'vies', 'celles', 'enfants', 'pays', 'libre', 'pays', 'vit', 'peuple']",dites vérité situation jugée catastrophique citoyen répondre non gouvernement entends marre faux semblant voulez démocratie renforce ecoutez citoyen cessez sourde oreille toutes mots dire gestion vies celles enfants pays libre pays vit peuple,dites vérité situation jugée catastrophique citoyen répondre non gouvernement entends marre faux semblant voulez démocratie renforce ecoutez citoyen cessez sourd oreille toutes mots dire gestion vie celles enfant pays libre pays vit peuple
1,1,17 965,DC11,Rendre la politique intéressante,Démocratie et citoyenneté,"['Rendre', 'la', 'politique', 'intéressante']","['rendre', 'politique', 'intéressante']",rendre politique intéressante,rendre politique intéressante
2,2,17 971,DC11,"Plus d'éducation civique, y compris au lycée. Proposer des formations en ligne pour mieux comprendre le fonctionnement de notre état.",Démocratie et citoyenneté,"['Plus', ""d'éducation"", 'civique', ',', 'y', 'compris', 'au', 'lycée', '.', 'Proposer', 'des', 'formations', 'en', 'ligne', 'pour', 'mieux', 'comprendre', 'le', 'fonctionnement', 'de', 'notre', 'état', '.']","['plus', 'éducation', 'civique', 'compris', 'lycée', 'proposer', 'formations', 'ligne', 'mieux', 'comprendre', 'fonctionnement']",plus éducation civique compris lycée proposer formations ligne mieux comprendre fonctionnement,plus éducation civique compris lycée proposer formation ligne mieux comprendre fonctionnement
3,3,17 974,DC11,Information du citoyen + vote oblugatoire,Démocratie et citoyenneté,"['Information', 'du', 'citoyen', '+', 'vote', 'oblugatoire']","['information', 'citoyen', 'vote', 'oblugatoire']",information citoyen vote oblugatoire,information citoyen vote oblugatoire
4,4,18 019,DC11,Qu'il arrête leur connerie comme les 80km/h,Démocratie et citoyenneté,"[""Qu'il"", 'arrête', 'leur', 'connerie', 'comme', 'les', '80km/h']","['arrête', 'connerie', 'km']",arrête connerie km,arrête connerie kilomètre


In [9]:
sw = stop_words.get_stop_words(language='fr')

In [10]:
# listes des mots à retirer du tf_idf
stopwords_additionnel = [
    'les',
    'a',
    'plus',
    'faut',
    'tout',
    'tous',
    'prends',
    'tre',
    'si',
    'non',
    'doit',
    'avoir',
    'comme',
    'trop',
    'leurs',
    'faire',
    'ils',
    'peut',
    'bien',
    'aussi',
    'cela',
    'gens',
    'sans',
    'car',
    'très',
    'fait',
    '\'',
    'qu\'',
    'd\'',
    'l\'',
    '"',
    'être',
    'mai',
    'faudrait'
]
stop_words_complete = sw + stopwords.words('french') + stopwords_additionnel
stop_words_complete = list(set(stop_words_complete))

## Analyse thème contribution libre

In [18]:
vectorizer = CountVectorizer(max_df=0.95, min_df=20, stop_words=stop_words_complete)
%time data_vectorized = vectorizer.fit_transform(answers[answers['theme'] == "Contribution libre"]['lemmatized_answer'].values.astype('U'))
data_vectorized.shape

CPU times: user 60.5 ms, sys: 8.3 ms, total: 68.8 ms
Wall time: 72.6 ms


(581, 158)

In [20]:
# Define Search Param
search_params = {'n_components': [3, 4, 5], 'learning_decay': [.5, .9], 'max_iter': [30]}

# Init the Model
lda = LatentDirichletAllocation(max_iter=10, learning_method='batch', verbose=True)

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params, verbose=0)

# Do the Grid Search
model.fit(data_vectorized)

/anaconda3/envs/elfrance/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


iteration: 1 of max_iter: 30
iteration: 2 of max_iter: 30
iteration: 3 of max_iter: 30
iteration: 4 of max_iter: 30
iteration: 5 of max_iter: 30
iteration: 6 of max_iter: 30
iteration: 7 of max_iter: 30
iteration: 8 of max_iter: 30
iteration: 9 of max_iter: 30
iteration: 10 of max_iter: 30
iteration: 11 of max_iter: 30
iteration: 12 of max_iter: 30
iteration: 13 of max_iter: 30
iteration: 14 of max_iter: 30
iteration: 15 of max_iter: 30
iteration: 16 of max_iter: 30
iteration: 17 of max_iter: 30
iteration: 18 of max_iter: 30
iteration: 19 of max_iter: 30
iteration: 20 of max_iter: 30
iteration: 21 of max_iter: 30
iteration: 22 of max_iter: 30
iteration: 23 of max_iter: 30
iteration: 24 of max_iter: 30
iteration: 25 of max_iter: 30
iteration: 26 of max_iter: 30
iteration: 27 of max_iter: 30
iteration: 28 of max_iter: 30
iteration: 29 of max_iter: 30
iteration: 30 of max_iter: 30
iteration: 1 of max_iter: 30
iteration: 2 of max_iter: 30
iteration: 3 of max_iter: 30
iteration: 4 of max_it

iteration: 9 of max_iter: 30
iteration: 10 of max_iter: 30
iteration: 11 of max_iter: 30
iteration: 12 of max_iter: 30
iteration: 13 of max_iter: 30
iteration: 14 of max_iter: 30
iteration: 15 of max_iter: 30
iteration: 16 of max_iter: 30
iteration: 17 of max_iter: 30
iteration: 18 of max_iter: 30
iteration: 19 of max_iter: 30
iteration: 20 of max_iter: 30
iteration: 21 of max_iter: 30
iteration: 22 of max_iter: 30
iteration: 23 of max_iter: 30
iteration: 24 of max_iter: 30
iteration: 25 of max_iter: 30
iteration: 26 of max_iter: 30
iteration: 27 of max_iter: 30
iteration: 28 of max_iter: 30
iteration: 29 of max_iter: 30
iteration: 30 of max_iter: 30
iteration: 1 of max_iter: 30
iteration: 2 of max_iter: 30
iteration: 3 of max_iter: 30
iteration: 4 of max_iter: 30
iteration: 5 of max_iter: 30
iteration: 6 of max_iter: 30
iteration: 7 of max_iter: 30
iteration: 8 of max_iter: 30
iteration: 9 of max_iter: 30
iteration: 10 of max_iter: 30
iteration: 11 of max_iter: 30
iteration: 12 of max

/anaconda3/envs/elfrance/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


iteration: 1 of max_iter: 30
iteration: 2 of max_iter: 30
iteration: 3 of max_iter: 30
iteration: 4 of max_iter: 30
iteration: 5 of max_iter: 30
iteration: 6 of max_iter: 30
iteration: 7 of max_iter: 30
iteration: 8 of max_iter: 30
iteration: 9 of max_iter: 30
iteration: 10 of max_iter: 30
iteration: 11 of max_iter: 30
iteration: 12 of max_iter: 30
iteration: 13 of max_iter: 30
iteration: 14 of max_iter: 30
iteration: 15 of max_iter: 30
iteration: 16 of max_iter: 30
iteration: 17 of max_iter: 30
iteration: 18 of max_iter: 30
iteration: 19 of max_iter: 30
iteration: 20 of max_iter: 30
iteration: 21 of max_iter: 30
iteration: 22 of max_iter: 30
iteration: 23 of max_iter: 30
iteration: 24 of max_iter: 30
iteration: 25 of max_iter: 30
iteration: 26 of max_iter: 30
iteration: 27 of max_iter: 30
iteration: 28 of max_iter: 30
iteration: 29 of max_iter: 30
iteration: 30 of max_iter: 30


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=True),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_components': [3, 4, 5], 'learning_decay': [0.5, 0.9], 'max_iter': [30]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [21]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.9, 'max_iter': 30, 'n_components': 3}
Best Log Likelihood Score:  -12006.353750560787
Model Perplexity:  148.629891430917


In [23]:
# Latent Dirichlet Allocation Model
NUM_TOPICS = 3
LEARNING_DECAY = 0.9
MAX_ITER = 30
lda = LatentDirichletAllocation(learning_decay=LEARNING_DECAY, n_components=NUM_TOPICS, max_iter=MAX_ITER, learning_method='batch',verbose=True)
data_lda = lda.fit_transform(data_vectorized)

iteration: 1 of max_iter: 30
iteration: 2 of max_iter: 30
iteration: 3 of max_iter: 30
iteration: 4 of max_iter: 30
iteration: 5 of max_iter: 30
iteration: 6 of max_iter: 30
iteration: 7 of max_iter: 30
iteration: 8 of max_iter: 30
iteration: 9 of max_iter: 30
iteration: 10 of max_iter: 30
iteration: 11 of max_iter: 30
iteration: 12 of max_iter: 30
iteration: 13 of max_iter: 30
iteration: 14 of max_iter: 30
iteration: 15 of max_iter: 30
iteration: 16 of max_iter: 30
iteration: 17 of max_iter: 30
iteration: 18 of max_iter: 30
iteration: 19 of max_iter: 30
iteration: 20 of max_iter: 30
iteration: 21 of max_iter: 30
iteration: 22 of max_iter: 30
iteration: 23 of max_iter: 30
iteration: 24 of max_iter: 30
iteration: 25 of max_iter: 30
iteration: 26 of max_iter: 30
iteration: 27 of max_iter: 30
iteration: 28 of max_iter: 30
iteration: 29 of max_iter: 30
iteration: 30 of max_iter: 30


In [24]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, data_vectorized, vectorizer, mds='tsne')
dash

/anaconda3/envs/elfrance/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                 x           y  topics  cluster       Freq
topic                                                     
0     -632.045471  -384.325073  1       1        36.903134
1     -1209.665405  212.164124  2       1        34.472260
2     -1437.453247 -586.322510  3       1        28.624606, topic_info=    Category        Freq          Term       Total  loglift  logprob
124  Default  108.000000  salaire       108.000000  30.0000  30.0000
67   Default  94.000000   impôt         94.000000   29.0000  29.0000
149  Default  92.000000   vie           92.000000   28.0000  28.0000
110  Default  86.000000   question      86.000000   27.0000  27.0000
115  Default  83.000000   revenu        83.000000   26.0000  26.0000
46   Default  90.000000   enfant        90.000000   25.0000  25.0000
156  Default  67.000000   élu           67.000000   24.0000  24.0000
101  Default  94.000000   pouvoir       94.000000   23.0000  23.0000
95   Default  53.000000   peuple        53.000000   22.0000  22.0000
70   Default  55.000000   isf           55.000000   21.0000  21.0000
144  Default  85.000000   travail       85.000000   20.0000  20.0000
60   Default  117.000000  français      117.000000  19.0000  19.0000
99   Default  41.000000   politiques    41.000000   18.0000  18.0000
92   Default  49.000000   pense         49.000000   17.0000  17.0000
117  Default  48.000000   ric           48.000000   16.0000  16.0000
91   Default  83.000000   pays          83.000000   15.0000  15.0000
113  Default  84.000000   retraite      84.000000   14.0000  14.0000
132  Default  35.000000   société       35.000000   13.0000  13.0000
72   Default  34.000000   jeune         34.000000   12.0000  12.0000
155  Default  45.000000   éducation     45.000000   11.0000  11.0000
133  Default  33.000000   souhaite      33.000000   10.0000  10.0000
77   Default  32.000000   maison        32.000000   9.0000   9.0000 
13   Default  41.000000   augmenter     41.000000   8.0000   8.0000 
12   Default  41.000000   augmentation  41.000000   7.0000   7.0000 
59   Default  157.000000  france        157.000000  6.0000   6.0000 
1    Default  37.000000   achat         37.000000   5.0000   5.0000 
125  Default  30.000000   santé         30.000000   4.0000   4.0000 
30   Default  36.000000   chose         36.000000   3.0000   3.0000 
42   Default  30.000000   démocratie    30.000000   2.0000   2.0000 
61   Default  55.000000   gouvernement  55.000000   1.0000   1.0000 
..       ...        ...            ...        ...      ...      ... 
54   Topic3   22.738097   femme         23.441316   1.2204  -4.4391 
149  Topic3   83.450638   vie           92.778308   1.1449  -3.1389 
110  Topic3   77.410958   question      86.917831   1.1351  -3.2140 
152  Topic3   29.324202   école         34.193128   1.0973  -4.1847 
155  Topic3   39.188584   éducation     45.917237   1.0924  -3.8947 
46   Topic3   76.945133   enfant        90.820075   1.0851  -3.2200 
86   Topic3   34.921243   niveau        43.968267   1.0205  -4.0100 
7    Topic3   23.174592   année         29.312017   1.0160  -4.4201 
79   Topic3   16.776508   meilleur      21.490554   1.0033  -4.7432 
85   Topic3   17.634962   nationale     23.451216   0.9659  -4.6932 
21   Topic3   23.482797   banque        31.247795   0.9652  -4.4069 
8    Topic3   20.508508   après         27.341694   0.9633  -4.5423 
145  Topic3   19.320457   travailler    26.364708   0.9400  -4.6020 
57   Topic3   19.113520   fonction      26.384168   0.9285  -4.6127 
32   Topic3   19.622477   condition     27.361597   0.9184  -4.5865 
94   Topic3   37.044212   petit         52.771786   0.8970  -3.9510 
144  Topic3   58.524082   travail       85.023631   0.8774  -3.4937 
36   Topic3   33.030828   doivent       49.843704   0.8395  -4.0657 
88   Topic3   18.142797   normal        28.333095   0.8051  -4.6649 
52   Topic3   20.445276   européen      32.230684   0.7957  -4.5454 
6    Topic3   56.809349   an            91.808380   0.7709  -3.5234 
113  